# Support codes for the article: TITLE

Please, if you use any information present in this notebook use the citation:

Hasselwander, M. and Tamagusko, A. (2020). TITLE. Retrieved from https://github.com/tamagusko/covid19/manila (accessed on MONTH DAY, YEAR). **Atualize to journal citation.**  

# Basic info
© Tiago Tamagusko (tamagusko@gmail.com)  
Version: [CHANGELOG.md](/CHANGELOG.md)  
Project Page: <a href="https://github.com/tamagusko/covid19/manila">https://github.com/tamagusko/covid19/manila</a>  
Licence: [LICENSE](/LICENSE)

## Import modules

In [12]:
import warnings
import datetime as dt
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

from matplotlib import style

warnings.simplefilter(action='ignore')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

# system variables
TODAY = dt.datetime.now().strftime('%y%m%d')
COUNTRY = 'Philippines'
REGION = 'Metro Manila'

## Data colection [1]

### Download dataset

In [13]:
# source: https://www.google.com/covid19/mobility/

mobiDay = pd.read_csv('data/Global_Mobility_Report.csv',
                      index_col='date',
                      parse_dates=[4],
                      infer_datetime_format=True)

### Cleaning data

In [14]:
mobiDay = mobiDay[mobiDay['country_region'] == COUNTRY].fillna(value=-1)
mobiDay = mobiDay[mobiDay['sub_region_1'] == REGION]

mobiDay.drop(mobiDay.columns[[0, 1, 2, 3, 4, 5]], axis=1, inplace=True)

### Rename columns

In [15]:
categories = ['retail_and_recreation',
              'grocery_and_pharmacy',
              'parks',
              'transit_stations',
              'workplaces',
              'residential']
mobiDay.columns = categories

### Quick view of data

In [16]:
mobiDay.describe().T

,count,mean,std,min,25%,50%,75%,max
retail_and_recreation,142.0,-61.683099,31.247438,-92.0,-85.00,-78.0,-57.00,8.0
grocery_and_pharmacy,142.0,-34.274648,23.417016,-80.0,-57.00,-37.0,-21.00,10.0
parks,142.0,-62.330986,33.037851,-93.0,-83.75,-78.0,-55.25,19.0
transit_stations,142.0,-66.640845,31.754676,-95.0,-90.00,-82.0,-66.00,5.0
workplaces,142.0,-52.753521,29.913394,-88.0,-77.75,-61.5,-40.25,7.0
residential,142.0,26.485915,14.005438,0.0,22.00,29.0,38.00,47.0


### Split data by period (lockdown and non-lockdown)

*Note: The lockdown period is not real, I created it just to make the code. In the future we have to fix this.*

In [17]:
# these periods are not real

lockdownBegin = '2020-03-17'  # first day of lockdown
lockdownEnd = '2020-04-18'  # last day of lockdown

beforeLockdown = mobiDay[:lockdownBegin]  # all data before lockdown
afterLockdown = mobiDay[lockdownEnd:]  # all data after lockdown

lockdownData = mobiDay[lockdownBegin:lockdownEnd]

nonLockdownData = pd.concat([beforeLockdown[:-1], afterLockdown[1:]], axis=0)

### saving average values for periods

In [18]:
lockdownMedian = []
for column in lockdownData:
    lockdownMedian.append(lockdownData[column].median(axis=0, skipna=True))

nonLockdownMedian = []
for column in nonLockdownData:
    nonLockdownMedian.append(
        nonLockdownData[column].median(axis=0, skipna=True))

In [19]:
lockdownMedian

[-84.0, -60.0, -79.0, -91.0, -80.0, 39.0]

In [20]:
nonLockdownMedian

[-65.0, -25.0, -77.0, -69.0, -55.0, 27.0]

In [21]:
# residential data excluded

data = pd.DataFrame({'categories': categories,
                     'lockdown': lockdownMedian, 
                     'non-lockdown': nonLockdownMedian})

data.set_index('categories')

,lockdown,non-lockdown
categories,,
retail_and_recreation,-84.0,-65.0
grocery_and_pharmacy,-60.0,-25.0
parks,-79.0,-77.0
transit_stations,-91.0,-69.0
workplaces,-80.0,-55.0
residential,39.0,27.0


In [22]:
px.scatter(data, x='lockdown', y='non-lockdown', color='categories')

## References
[1] Google LLC. Google COVID-19 Community Mobility Reports. Available online: https://www.google.com/covid19/mobility/ (accessed on July 12, 2020).